### CVAT download dataset

In [7]:
import mlflow
import os
import pandas as pd
import requests
import shutil
import sys
import hashlib
sys.path.insert(0, "..")
from zipfile import ZipFile

from datetime import datetime
from glob import glob
from io import BytesIO
from pathlib import Path
from pytorch_lightning import  Trainer


In [3]:
URL = 'http://10.72.125.105:8080'
API_URL = '%s/api/v1' % URL
LOGIN = 'ml'
PASSWORD = 'NuAzu8mO'
PROJECT = 'Liza'

def get_session(url, username, password):
    session = requests.Session()
    response=session.post(url, data={'username':username,'password':password})
    response.raise_for_status()
    if 'csrftoken' in response.cookies:
        session.headers['X-CSRFToken'] = response.cookies['csrftoken']
    return session

def get_request(self, method, query=None):
    url = "%s/%s" % (API_URL,method)
    if ~(query is None):
        url+='?%s'%query
    request = self.get(url)
    if request.headers['Content-Type'] == 'application/json':
        return request.json()
    else:
        return request

In [4]:
url = '%s/auth/login' % API_URL
session = get_session(url, LOGIN, PASSWORD)
projects = get_request(session,'projects','?search=%s' % (PROJECT))
project= projects['results'][0]

In [59]:
def makePath(path):
    if path.exists() and path.is_dir():
        shutil.rmtree(path)
    os.mkdir(path)

In [61]:
dir_path = Path(os.path.join('..','tmp'))
train_path = Path(os.path.join('..','dataset','objects'))

makePath(dir_path)
makePath(train_path)

for task in project['tasks']:
    task_id = task['id']
    chunk_size = task['data_chunk_size']
    annotations = get_request(session,'tasks/%s/annotations' % (task_id))
    shapes = pd.DataFrame(annotations['shapes'])
    chunk_number=0

    for chunk_n in range((shapes.frame.max() // chunk_size) + 1):
        start_chunk = chunk_n * chunk_size
        chunk_shapes = shapes[(shapes.frame >= start_chunk) & (shapes.frame < start_chunk+chunk_size)]

        if len(chunk_shapes) > 0:
            data=get_request(session,'tasks/%s/data' % task_id, 'type=chunk&number=%s&quality=original' % chunk_number)
            zipfile= ZipFile(BytesIO(data.content))
            zipfile.extractall(dir_path)

            for frame in chunk_shapes.frame.unique():
                filename = '%s.jpg' % str(1000000 + (frame-start_chunk))[1:]   
                m = hashlib.md5()

                for line in open(dir_path.joinpath(filename), 'rb'):
                    m.update(line)

                new_filename = ('%s.%s' % (m.hexdigest(), filename.split('.')[-1]))
                shapes.loc[shapes['frame']==frame,'md5']=m.hexdigest()
                shutil.move(dir_path.joinpath(filename),train_path.joinpath(new_filename))

shutil.rmtree(dir_path)

In [64]:
shapes[['points','md5']].to_csv(train_path.joinpath('data.csv'),index=False)